In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import torch
from torch import nn
from importlib.util import find_spec
if find_spec("text_recognizer") is None:
    import sys
    sys.path.append('..')
    

In [2]:
from hydra import compose, initialize
from omegaconf import OmegaConf
from hydra.utils import instantiate

In [3]:
# context initialization
with initialize(config_path="../training/conf/", job_name="test_app"):
    cfg = compose(config_name="config", overrides=["+experiment=vqvae"])
    print(OmegaConf.to_yaml(cfg))
    print(cfg)

callbacks:
  model_checkpoint:
    _target_: pytorch_lightning.callbacks.ModelCheckpoint
    monitor: val/loss
    save_top_k: 1
    save_last: true
    mode: min
    verbose: false
    dirpath: checkpoints/
    filename: '{epoch:02d}'
  learning_rate_monitor:
    _target_: pytorch_lightning.callbacks.LearningRateMonitor
    logging_interval: step
    log_momentum: false
  watch_model:
    _target_: callbacks.wandb_callbacks.WatchModel
    log: all
    log_freq: 100
  upload_code_as_artifact:
    _target_: callbacks.wandb_callbacks.UploadCodeAsArtifact
    project_dir: ${work_dir}/text_recognizer
  upload_ckpts_as_artifact:
    _target_: callbacks.wandb_callbacks.UploadCheckpointsAsArtifact
    ckpt_dir: checkpoints/
    upload_best_only: true
  log_image_reconstruction:
    _target_: callbacks.wandb_callbacks.LogReconstuctedImages
    num_samples: 8
criterion:
  _target_: torch.nn.MSELoss
  reduction: mean
datamodule:
  _target_: text_recognizer.data.iam_extended_paragraphs.IAMExtende

In [4]:
mapping = instantiate(cfg.mapping)

2021-08-04 04:49:04.188 | DEBUG    | text_recognizer.data.word_piece_mapping:__init__:37 - Using data dir: /home/aktersnurra/projects/text-recognizer/data/downloaded/iam/iamdb


In [35]:
network = instantiate(cfg.network)

In [36]:
x = torch.rand(1, 1, 576, 640)

In [37]:
network.encode(x)[0].shape

torch.Size([1, 64, 144, 160])

In [38]:
network(x)[0].shape

torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])


torch.Size([1, 1, 576, 640])

In [ ]:
576 / 4

In [ ]:
t = torch.randint(0, 1006, (1, 451)).cuda()

In [ ]:
z = torch.rand((1, 36 * 40, 128)).cuda()

In [ ]:
network = network.cuda()

In [ ]:
network.decode(z, t).shape

In [ ]:
decoder = decoder.cuda()

In [ ]:
decoder(z, t).shape

In [ ]:
OmegaConf.set_struct(cfg, False)

In [ ]:
datamodule = instantiate(cfg.datamodule, mapping=mapping)

In [ ]:
datamodule.prepare_data()
datamodule.setup()

In [ ]:
len(datamodule.train_dataloader())

In [ ]:
mapping

In [ ]:
config = cfg

In [ ]:
loss_fn = instantiate(cfg.criterion)

In [ ]:
import hydra

In [ ]:
    model = hydra.utils.instantiate(
        config.model,
        mapping=mapping,
        network=network,
        loss_fn=loss_fn,
        optimizer_config=config.optimizer,
        lr_scheduler_config=config.lr_scheduler,
        _recursive_=False,
    )


In [ ]:
mapping.get_index

In [ ]:
net = instantiate(cfg)

In [ ]:
net

In [ ]:
img = torch.rand(4, 1, 576, 640)

In [ ]:
y = torch.randint(0, 1006, (4, 451))

In [ ]:
y.shape

In [ ]:
net = net.cuda()
img = img.cuda()
y = y.cuda()

In [ ]:
net(img, y).shape

In [ ]:
from torchsummary import summary

In [ ]:
summary(net, [(1, 576, 640), (451,)], device="cpu", depth=2)